In [ ]:
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa

# Augment training data
def augment_training_data(images, labels):

    expanded_images = []
    expanded_labels = []

    j = 0 # counter
    for x, y in zip(images, labels):
        j = j+1
        if j%500==0:
            print ('expanding data : %03d / %03d' % (j,np.size(images,0)))
        
        # register original data
        expanded_images.append(x)
        expanded_labels.append(y)
        
        new_img_ = seq.augment_images(x)
        
        # register new training data
        expanded_images.append(new_img_)
        expanded_labels.append(y)
               
    return expanded_images, expanded_labels 

In [11]:
from imgaug import augmenters as iaa

seq = iaa.Sequential([
    iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
])

for batch_idx in range(1000):
    # 'images' should be either a 4D numpy array of shape (N, height, width, channels)
    # or a list of 3D numpy arrays, each having shape (height, width, channels).
    # Grayscale images must have shape (height, width, 1) each.
    # All images must have numpy's dtype uint8. Values are expected to be in
    # range 0-255.
    images = load_batch(batch_idx)
    images_aug = seq.augment_images(images)
    train_on_images(images_aug)

NameError: name 'load_batch' is not defined

In [13]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
from skimage import data
from scipy import misc

# Example augmentation sequence to run in the background.
augseq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.CoarseDropout(p=0.1, size_percent=0.1)
])

# A generator that loads batches from the hard drive.
def load_batches():
    # Here, load 10 batches of size 4 each.
    # You can also load an infinite amount of batches, if you don't train
    # in epochs.
    batch_size = 4
    nb_batches = 10

    # Here, for simplicity we just always use the same image.
    astronaut = data.astronaut()
    astronaut = ia.imresize_single_image(astronaut, (64, 64))

    for i in range(nb_batches):
        # A list containing all images of the batch.
        batch_images = []
        # A list containing IDs of images in the batch. This is not necessary
        # for the background augmentation and here just used to showcase that
        # you can transfer additional information.
        batch_data = []

        # Add some images to the batch.
        for b in range(batch_size):
            batch_images.append(astronaut)
            batch_data.append((i, b))

        # Create the batch object to send to the background processes.
        batch = ia.Batch(
            images=np.array(batch_images, dtype=np.uint8),
            data=batch_data
        )

        yield batch

# background augmentation consists of two components:
#  (1) BatchLoader, which runs in a Thread and calls repeatedly a user-defined
#      function (here: load_batches) to load batches (optionally with keypoints
#      and additional information) and sends them to a queue of batches.
#  (2) BackgroundAugmenter, which runs several background processes (on other
#      CPU cores). Each process takes batches from the queue defined by (1),
#      augments images/keypoints and sends them to another queue.
# The main process can then read augmented batches from the queue defined
# by (2).
batch_loader = ia.BatchLoader(load_batches)
bg_augmenter = ia.BackgroundAugmenter(batch_loader, augseq)

# Run until load_batches() returns nothing anymore. This also allows infinite
# training.
while True:
    print("Next batch...")
    batch = bg_augmenter.get_batch()
    if batch is None:
        print("Finished epoch.")
        break
    images_aug = batch.images_aug

    print("Image IDs: ", batch.data)

    misc.imshow(np.hstack(list(images_aug)))

batch_loader.terminate()
bg_augmenter.terminate()

Next batch...
Image IDs:  [(1, 0), (1, 1), (1, 2), (1, 3)]


RuntimeError: Could not execute image viewer.